
 集成学习的特性使其应用到不平衡学习是很自然的选择，接下来主要对基于样本采样技术的集成学习做介绍
 
 ### 一.Bagging
 
BalancedBagging的过程可以简单描述如下：   

![avatar](./source/asBagging.png)



In [10]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.ensemble import BalancedBaggingClassifier 

In [11]:
X, y = make_classification(n_classes=2, class_sep=2,
weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0,
n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0)
bbc = BalancedBaggingClassifier(random_state=42)
bbc.fit(X_train, y_train) 

BalancedBaggingClassifier(base_estimator=None, bootstrap=True,
                          bootstrap_features=False, max_features=1.0,
                          max_samples=1.0, n_estimators=10, n_jobs=1,
                          oob_score=False, random_state=42, ratio=None,
                          replacement=False, sampling_strategy='auto',
                          verbose=0, warm_start=False)

In [13]:
y_pred = bbc.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 23,   0],
       [  2, 225]], dtype=int64)

有时离群样本点往往会坏在某些维度上(取值特别高/特别低)，可以借用RandomForest的思路再对特征做一个随机抽取，生成不同的随机子空间(random subspace,后面简称RS),这时流程如下：   
![avatar](./source/asBagging_FS.png)

In [14]:
from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(max_depth=2, random_state=0)
brfc.fit(X_train, y_train) 

BalancedRandomForestClassifier(bootstrap=True, class_weight=None,
                               criterion='gini', max_depth=2,
                               max_features='auto', max_leaf_nodes=None,
                               min_impurity_decrease=0.0, min_samples_leaf=2,
                               min_samples_split=2,
                               min_weight_fraction_leaf=0.0, n_estimators=100,
                               n_jobs=1, oob_score=False, random_state=0,
                               replacement=False, sampling_strategy='auto',
                               verbose=0, warm_start=False)

In [15]:
y_pred = brfc.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 23,   0],
       [  2, 225]], dtype=int64)

In [16]:
bbc = BalancedBaggingClassifier(random_state=42,bootstrap_features=True)
bbc.fit(X_train, y_train) 

BalancedBaggingClassifier(base_estimator=None, bootstrap=True,
                          bootstrap_features=True, max_features=1.0,
                          max_samples=1.0, n_estimators=10, n_jobs=1,
                          oob_score=False, random_state=42, ratio=None,
                          replacement=False, sampling_strategy='auto',
                          verbose=0, warm_start=False)

In [17]:
y_pred = bbc.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 23,   0],
       [  2, 225]], dtype=int64)

**小结**：上面流程其实是框架性质的：  

（1）对于采样其实可以任意选择，比如随机欠采样，聚类采样等都可以；   

（2）对于RS其实还有改进空间，比如可以选择差异性较大的子空间，比如按相关性度量对特征维度进行聚类，然后从每个类中选择一个有代表性的特征出来构造新的特征子空间；另外lgb中特征选择方式也可以参考

### 二.Boosting
boosting比较有代表的就是RUSBoost和SMOTEBoost了，顾名思义，RUSBoost表示random undersampling+adaboost，而SMOTEBoost表示smote+adaboost  

**RUSBoost的流程如下：**   
![avatar](./source/RUSBoost.png)  


**SMOTEBoost的流程如下：**   

![avatar](./source/smoteboost.png)  


可见一个是对样本中多数类做欠采样，另一个是对少数类做过采样，其本质都是增加少数类的权重，其中样本权重的更新以及学习器的权重与adaboost一致

In [18]:
from imblearn.ensemble import RUSBoostClassifier
rbc = RUSBoostClassifier(random_state=0)
rbc.fit(X_train, y_train)
y_pred = rbc.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 23,   0],
       [  2, 225]], dtype=int64)

### 三.Bagging&Boosting

Bagging和Boosting也可以结合起来使用，比较常用的手段是Bagging嵌套Boosting，比较有代表的模型是EasyEnsemble以及BalancedCascade

**EasyEnsemble流程如下：**    

首先采用RUS随机生成多个平衡的训练子集，然后再在每个训练子集上分别训练一个AdaBoost，最后再对这些Adaboost做Bagging

![avatar](./source/easyensemble.svg)  


**BalanceCascade流程如下：**   

它是对EasyEnsemble的改进，每一个AdaBoost的训练会排除之前已经正确分类的多数类，所以BalanceCascade只能以串行的方式训练   

![avatar](./source/BalancedCascade.svg)  

注意：上面Ti表示bias

In [19]:
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=42)
eec.fit(X_train, y_train)
y_pred = eec.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 23,   0],
       [  2, 225]], dtype=int64)

### 四.其他集成方法

对于代价敏感学习以及决策输出补偿也可以尝试集成学习的方法，比如前面介绍的AdaCost可看做基于代价敏感学习的一种集成学习方法